**Submitted by `your_name` on `date_of_submission`**

# Optimization Exercises

This notebook was written by Selin Ataç (selin.atac@epfl.ch) and edited by Dr. Léa Ricard (lea.ricard@epfl.ch) for the Optimization and Simulation course at EPFL (https://edu.epfl.ch/studyplan/en/doctoral_school/civil-and-environmental-engineering/coursebook/optimization-and-simulation-MATH-600). 

Please contact before distributing or reusing the material below.

## Table of Contents
* [(Priced) Knapsack Problem](#(Priced)-Knapsack-problem)
   * [Problem definition and encoding](#Problem-definition-(KP))
   * [Implementation](#Implementation)
   * [Core functionalities](#The-core-functionalities)
* [Multi-objective optimization](#Multi-objective-optimization)
    * [Dominance](#Dominance)

## (Priced) Knapsack problem

### Problem definition (KP)

Each item has a weight $w$, utility $u$ and cost $c$. What is the set of items that can be put in the knapsack that is below the maximum weigh, minimizes the cost, and maximizes the utility?

### Problem encoding

We consecutively number the items: $0,..., n$

We define our decision variables as $x_i=1$ if item $i$ is in the knapsack, $0$ otherwise.

### Objectives

1. Maximizing the utility: $\max \mathcal{f}_1(x)=\sum_{i=1}^n u_i x_i$ 
2. Minimizing the cost: $\min \mathcal{f}_2(x)=\sum_{i=1}^n c_i x_i$
3. $w^Tx \leq W$ where $W$ is the maximum weight allowed, e.g. 100kg

### Implementation

#### The required python libraries
You will use the following python libraries in this exercise: `numpy`, `plotly`. Install it using `pip` on your command line:

    pip install numpy plotly

or if you are using conda:

    conda install numpy
    conda install -c plotly plotly

In [1]:
import numpy as np
import plotly
import plotly.graph_objects as go
import time

from numpy.random import Generator, PCG64 
from plotly.subplots import make_subplots

### The core functionalities

Next, we implement the functions `total_cost()`, `total_utility()`, and `weight()`:

In [2]:
def total_cost(sol, item_costs):
    """Function to implement

    Args:
        sol (list): a list of all decision variables x_1, ... x_n
        item_costs (list): a list of the cost c_1, ..., c_n of the items
        
    Returns:
        cost (float): total cost of the items in the knapsack
    
    """  
    # Implement your solution here
    cost = sum(x * c for x, c in zip(sol, item_costs))
    return cost

def total_utility(sol, item_utilities):
    """Function to implement

    Args:
        sol (list): a list of all decision variables x_1, ... x_n
        item_utilities (list): a list of the utility u_1, ..., u_n of the items
        
    Returns:
        cost (float): total utility of the items in the knapsack
    
    """  
    # Implement your solution here
    utility  = sum(x * u for x, u in zip(sol, item_utilities))
    return utility

def weight(sol, item_weights):
    """Function to implement

    Args:
        sol (list): a list of all decision variables x_1, ... x_n
        item_weights (list): a list of the weight w_1, ..., w_n of the items
        
    Returns:
        cost (float): total weight of the items in the knapsack
    
    """ 
    # Implement your solution here
    weight=sum(x * w for x, w in zip(sol, item_weights))
    return weight

#### Test the functions `total_cost()`, `total_utility()`, and `weight()`

In [3]:
rg = Generator(PCG64(42069)) # set your own unique seed number
n_items = 5
item_weights = rg.integers(10, 50, n_items)
item_utilities = rg.integers(0, 100, n_items)
item_costs = rg.random(n_items)

print("weights: ", item_weights)
print("utility: ", item_utilities)
print("cost: ", item_costs)
sol = rg.integers(0, 2, n_items) # Random solution
print("Knapsack composition: ", sol)
print("Total cost: ", total_cost(sol, item_costs))
print("Total utility: ", total_utility(sol, item_utilities))
print("Total weight: ", weight(sol, item_weights))

weights:  [45 47 29 21 20]
utility:  [ 7 98 32 82 55]
cost:  [0.86346747 0.72134798 0.69142444 0.54085333 0.45354493]
Knapsack composition:  [1 1 0 0 0]
Total cost:  1.5848154561081227
Total utility:  105
Total weight:  92


# Multi-objective optimization

In this lab, we will implement a multi-objective optimization algorithm (described in the lecture notes). Several methods are possible:

1. Weighted sum
2. Lexicographic rules (e.g., sequence by importance)
3. Constrained optimization
4. **Local search**/**heuristics**

Here, we will use a heurisric local search algorithm.



## Concepts in multi-objective optimization

- We need to minimize several objective functions.
- In many practical applications, the objectives are conflicting.
- Improving one objective may deteriorate several others (e.g., risk vs reward).

$$
min F(x) = (f_1(x), ..., f_p(x))
$$
subject to
$$
x \in F \subseteq \mathbb{R}^n
$$
where
$$
F: \mathbb{R}^n \rightarrow \mathbb{R}^p
$$

## Dominance

Notation:

$x_1$ dominates $x_2$: $F(x_1)\prec F(x_2)$

Dominance must fulfill two conditions, assuming we want to **minimize**

1. $x_1$ is no worse than $x_2$ in **any** objective: $\forall i \in \{1,...,p\},f_i(x_1)\leq f_i(x_2)$
2. $x_1$ is strictly better in at least one objective: $\exists i \in \{1,...,p\}, f_i(x_1)<f_i(x_2)$

### Pseudo algorithm for local search

Main difference with single objective optimization is that we need to maintain a set $P$ of potential Pareto optimal solutions such that $\forall x,y \in P, F(x) \nprec F(y)$ and $F(y) \nprec F(x)$.

**Initialization:**

Start with a first set $P$ of candidate solutions

**Main iteration:**

1. Select a random solution $x$ from $P$, define $x^+$ as a neighbour solution  
2. Define 2 sets: $D(x^+)$ and $S(x^+)$, where

$$D(x^+)=\{y\in P, s.t. F(x^+)\prec F(y)\}$$

$$S(x^+)=\{y\in P, s.t. F(y)\prec F(x^+)\}$$

In other words, at each iteration, compare each objective function of the new solution $x^+$ against all $y$ in $P$. If the condition is fulfiled for any of the above, put $y$ into the respective set.

3. If $S(x^+)=\emptyset$ (i.e., if $x^+$ is not dominated by any $y \in P$)
    
    Update $P$ (remove all exisiting solutions in the Pareto set, $P$, if it appears in $D$ and add the new solution to $P$):
$$
P^+ = P\cup \{x^+\} \setminus D(x^+)
$$

Implement the dominance rule and compute the sets $D$ and $S$ in functions `dominance()` and `generate_D_and_S()`.

In [4]:
def dominance(cost_x1, cost_x2):
    """Function to implement
    Determines dominance between two solutions based on their cost components with respect to two objectives.

    Args:
        cost_x1 (list): a list of cost components with respect to objectives f_1 and f_2 for solution x1.
        cost_x2 (list): a list of cost components with respect to objectives f_1 and f_2 for solution x2.
        
    Returns:
        bool: True if solution x1 dominates solution x2, False otherwise.
        
    Examples:
        >>> dominance([105, 1.58], [99, 2.25])
        True
        >>> dominance([105, 1.58], [99, 1.45])
        False
    """ 
    
    # Implement your solution here
    return (cost_x1[0] >= cost_x2[0] and cost_x1[1] <= cost_x2[1]) and (cost_x1[0] > cost_x2[0] or cost_x1[1] < cost_x2[1])

def generate_D_and_S(P, cost_new, item_utilities, item_costs):
    
    """Function to implement
    Generates sets of solutions dominated and dominating a new neighbor solution within a Pareto set.

    Args:
        P (dict): a dictionary containing solutions in a Pareto set.
        cost_new (list): a list of cost components with respect to objectives f_1 and f_2 for a new neighbor 
        solution.
        item_utilities (list): a list of the utility u_1, ..., u_n of the items.
        item_costs (list): a list of the cost c_1, ..., c_n of the items.

    Returns:
        D (dict): A set of solutions in P dominated by the new neighbor solution.
        S (dict): A set of solutions in P that dominate the new neighbor solution.
        
    """ 
        
    D = {}
    S = {}
    
    # Implement your solution here
    for sol, costs in P.items():
        if dominance(cost_new, costs):
            D[sol] = costs
        elif dominance(costs, cost_new):
            S[sol] = costs

    return D,S

### Neighbourhood structures.

Implement at least one neighborhood structure. Ensure that this neighborhood structure (or these neighborhood structures) enable(s) exploration of the Pareto frontier, from solutions with low costs and low utilities to solutions with high costs and high utilities. Ensure each neighborhood structure outputs feasible solutions by verifying that the total weight does not exceed the given maximum weight, $W$.

In [5]:
def neighborhood_i(solution, item_weights, item_utilities, item_costs, W):
    """Function to implement
    Generate a neighbor solution within the neighborhood structure.
    Args:
        solution (list): a list of all decision variables x_1, ... x_n of a current solution.
        item_weights (list): a list of the weight w_1, ..., w_n of the items.
        item_utilities (list): a list of the utility u_1, ..., u_n of the items.
        item_costs (list): a list of the cost c_1, ..., c_n of the items.
        W (int): maximum weight of the knapsack.

    Returns:
        neighbor (list): a list of all decision variables x_1, ... x_n of a neighboring solution.
    """ 
    
    # Implement your solution here
    neighbor = solution.copy()
    
    
    # Decide how many bits to flip
    num_item_to_flip = np.random.randint(1, 4)  #change the decision to take or not item for few items
    indices = np.random.choice(len(solution), num_item_to_flip, replace=False)
    
    for i in indices:
        neighbor[i] = 1 - neighbor[i]
    
    if weight(neighbor, item_weights) > W:
        # We check the weight constraint and change decision to take or not an item if the condition is not met 
        indices = np.random.permutation(len(solution))
        for i in indices:
            if neighbor[i] == 1:
                neighbor[i] = 0
                if weight(neighbor, item_weights) <= W:
                    break
    
    return neighbor

### Main loop


Generate an initial random solution and add it to P. For example, start with 2 random items from the set.


Main iteration

    a. Select a random solution from set $P$
    b. Find neighbor(s)
    c. Compute the new cost of the new solution(s)
    d. Apply Pareto dominance, i.e., sets $D$ and $S$
    e. If $S$ is empty, update $P$

        

In [6]:
def main(item_weights, item_utilities, item_costs, iterations=100, W=400):
    """Solves a multi-objective knapsack-like problem using Pareto optimization.

    Args:
        item_weights (list): weights of the items.
        item_utilities (list): utility values of the items.
        item_costs (list): cost values of the items.
        iterations (int): number of optimization iterations.
        W (int): maximum allowed weight.

    Returns:
        P (dict): dictionary containing non-dominated (Pareto) solutions.
        D_all (dict): dictionary containing all dominated solutions encountered.
    """

    def generate_initial_solution(n, num_items=2):
        """Generate a single initial random solution with `num_items` set to 1."""
        sol = [0] * n
        selected_items = np.random.choice(range(n), num_items, replace=False)
        for item in selected_items:
            sol[item] = 1
        return sol

    P = {}
    D_all = {}

    rg = Generator(PCG64(42069))
    n_items = len(item_weights)

    # Generate valid initial solution
    initial_solution = generate_initial_solution(n_items)
    while weight(initial_solution, item_weights) > W:
        initial_solution = generate_initial_solution(n_items)

    initial_cost = [total_utility(initial_solution, item_utilities), total_cost(initial_solution, item_costs)]
    P[tuple(initial_solution)] = initial_cost

    for _ in range(iterations):
        # Select a random solution from P
        keys = list(P.keys())
        selected_index = rg.choice(len(keys))
        current_solution = list(keys[selected_index])

        # Generate a neighbor
        neighbor_solution = neighborhood_i(current_solution, item_weights, item_utilities, item_costs, W)

        # Compute the new cost of the new solution(s)
        neighbor_cost = [total_utility(neighbor_solution, item_utilities), total_cost(neighbor_solution, item_costs)]

        # Apply Pareto dominance
        D, S = generate_D_and_S(P, neighbor_cost, item_utilities, item_costs)

        # Update Pareto set
        if not S:
            for sol in D:
                del P[sol]
            P[tuple(neighbor_solution)] = neighbor_cost
            D_all.update(D)

    return P, D_all

### Run the optimization

In [7]:
rg = Generator(PCG64(42069)) # set your own unique seed number
n_items = 20
item_weights = rg.integers(10, 50, n_items)
item_utilities = rg.integers(0, 100, n_items)
item_costs = rg.random(n_items)
P, D_all = main(item_weights, item_utilities, item_costs, iterations=1000, W=300) # weight limit=W

### View the Pareto frontier

In [8]:
def draw_pareto_frontier(P, D_all):
    costs_P = []
    utilities_P = []
    costs_D = []
    utilities_D = []

    for sol, cost in P.items():
        utilities_P.append(cost[0])
        costs_P.append(cost[1])

    for sol, cost in D_all.items():
        utilities_D.append(cost[0])
        costs_D.append(cost[1])

    trace_P = go.Scatter(x=utilities_P, y=costs_P, mode='markers', name='Non-dominated')
    trace_D = go.Scatter(x=utilities_D, y=costs_D, mode='markers', name='Dominated', marker=dict(symbol='x'))

    layout = go.Layout(
        template="presentation",
        xaxis=dict(title='Utility'),
        yaxis=dict(title='Cost'),
        legend=dict(x=0, y=1),
        width=800, 
        height=600
    )

    fig = go.Figure(data=[trace_P, trace_D], layout=layout)
    fig.show()


#### Run the function `draw_pareto_frontier()`

In [9]:
draw_pareto_frontier(P, D_all)

## Further exercises

Try to implement a full enumaration algorithm. Use the `itertools.product()` function to generate all possible solutions. Apply dominance conditions. 

Calculate the computational time and limitations of the full enumeration.

What is the maximum problem size (number of items) that you could solve with this approach?

In [10]:
from itertools import product

In [11]:
def knapsack_fe(item_weights, item_utilities, item_costs, W=400):
    """Function to implement

    Args:
        item_weights (list): a list of the weight w_1, ..., w_n of the items
        item_utilities (list): a list of the utility u_1, ..., u_n of the items.
        item_costs (list): a list of the cost c_1, ..., c_n of the items.
        W (int): maximum weight of the knapsack.

    Returns:
        P (dict): a dictionary containing solutions in a Pareto set.
        D_all (dict): a dictionary containing all dominated solutions.
        
    """ 
    
    P = {}
    D_all = {}
    
    # Implement your solution here
    n_items = len(item_weights)
    all_solutions = product([0, 1], repeat=n_items)
    
    for sol in all_solutions:
        if weight(sol, item_weights) <= W:
            sol_cost = [total_utility(sol, item_utilities), total_cost(sol, item_costs)]
            D, S = generate_D_and_S(P, sol_cost, item_utilities, item_costs)
            if not S:
                for s in D:
                    del P[s]
                P[tuple(sol)] = sol_cost
                D_all.update(D)
    
    return P, D_all

In [12]:
rg = Generator(PCG64(42069)) 
n_items = 20  # Define the number of items
item_weights = rg.integers(10, 50, n_items)
item_utilities = rg.integers(0, 100, n_items)
item_costs = rg.random(n_items)



In [13]:
P_fe, D_all_fe = knapsack_fe(item_weights, item_utilities, item_costs, W=300) # weight limit=W
draw_pareto_frontier(P_fe, D_all_fe)